# Web Scraping Gobierno Transparente Chile
##### Exploring wage expenses in the goberment. Public information.

Beautiful soup did not work. The server time outs when you connect through bsoup.

## Trying Selenium

In [16]:
import pandas as pd
import numpy as np
import unicodedata
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

import csv

import sys
sys.path.append('./lib/')

import functions
from functions import *
from tqdm import tnrange, tqdm_notebook
from unidecode import unidecode

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import lxml.html
import lxml

%load_ext autoreload

%autoreload 2

[autoreload of functions failed: Traceback (most recent call last):
  File "/home/jaime/anaconda3/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
  File "./lib/functions.py", line 86
    print 'Timeout Exception'
                            ^
SyntaxError: Missing parentheses in call to 'print'
]


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Gather data already scraped, before re-run

In [12]:
df_scraped_1 = pd.read_csv('./output/scraped_data_20170214.csv', encoding='latin_1', header=None, index_col=0, low_memory=False)
print(df_scraped_1.shape)
df_scraped_2 = pd.read_csv('./output/scraped_data_20170212.csv', encoding='latin_1', header=None, index_col=0, low_memory=False)
print(df_scraped_2.shape)
df_scraped_3 = pd.read_csv('./output/scraped_data_20170215.csv', encoding='latin_1', header=None, index_col=0, low_memory=False)
print(df_scraped_3.shape)
df_scraped_4 = pd.read_csv('./output/scraped_data_20170215-1.csv', encoding='latin_1', header=None, index_col=0, low_memory=False)
print(df_scraped_4.shape)
df_scraped_5 = pd.read_csv('./output/scraped_data_20170216.csv', encoding='latin_1', header=None, index_col=0, low_memory=False)
print(df_scraped_5.shape)
df_scraped_6 = pd.read_csv('./output/scraped_data_20170221.csv', encoding='latin_1', header=None, index_col=0, low_memory=False)
print(df_scraped_6.shape)

(44783, 21)
(25227, 21)
(50586, 20)
(3145, 20)
(36591, 20)
(15300, 20)


In [13]:
df_scraped = pd.concat([df_scraped_1, df_scraped_2, df_scraped_3, df_scraped_4, df_scraped_5, df_scraped_6]).drop_duplicates()
df_scraped.shape

(114551, 21)

In [34]:
df_visited = []

In [3]:
df_visited_1 = pd.read_csv('./output/log_opened_20170212.csv', header=None).ix[:,0].drop_duplicates().tolist()
df_visited_2 = pd.read_csv('./output/log_opened_20170214.csv', header=None).ix[:,0].drop_duplicates().tolist()
df_visited_3 = pd.read_csv('./output/log_opened_20170215.csv', header=None).ix[:,0].drop_duplicates().tolist()
df_visited_4 = pd.read_csv('./output/log_opened_20170221.csv', header=None).ix[:,0].drop_duplicates().tolist()
df_visited_5 = pd.read_csv('./output/log_opened_20170224.csv', header=None).ix[:,0].drop_duplicates().tolist()
df_visited_6 = pd.read_csv('./output/log_opened_20170225.csv', header=None).ix[:,0].drop_duplicates().tolist()


df_visited = list(set(df_visited_1 + df_visited_2 + df_visited_3 + df_visited_4 + df_visited_5 + df_visited_6))
print(len(df_visited))
print(len(df_visited_1))
print(len(df_visited_2))
print(len(df_visited_3))
print(len(df_visited_4))
print(len(df_visited_5))
print(len(df_visited_6))




14746
168
482
1797
156
1031
11130


# Main Scraping Run

In [17]:


output = './output/scraped_data_new.csv'

url = 'http://www.gobiernotransparentechile.cl/'

browser = webdriver.Chrome()
#browser = webdriver.Firefox()

df = getGovernmentData(output, url, browser, 0)

Entities:
['http://www.gobiernotransparentechile.cl/directorio/entidad/1', 'http://www.gobiernotransparentechile.cl/directorio/entidad/2', 'http://www.gobiernotransparentechile.cl/directorio/entidad/3', 'http://www.gobiernotransparentechile.cl/directorio/entidad/4', 'http://www.gobiernotransparentechile.cl/directorio/entidad/5', 'http://www.gobiernotransparentechile.cl/directorio/entidad/6', 'http://www.gobiernotransparentechile.cl/directorio/entidad/7', 'http://www.gobiernotransparentechile.cl/directorio/entidad/8', 'http://www.gobiernotransparentechile.cl/directorio/entidad/9', 'http://www.gobiernotransparentechile.cl/directorio/entidad/10', 'http://www.gobiernotransparentechile.cl/directorio/entidad/11', 'http://www.gobiernotransparentechile.cl/directorio/entidad/12', 'http://www.gobiernotransparentechile.cl/directorio/entidad/13', 'http://www.gobiernotransparentechile.cl/directorio/entidad/14', 'http://www.gobiernotransparentechile.cl/directorio/entidad/15', 'http://www.gobiernotra

Traceback (most recent call last):
  File "./lib/functions.py", line 62, in getDepartmentData
    div_years = browser.find_element_by_class_name("linksIntermedios")
  File "/home/jaime/anaconda3/lib/python3.6/site-packages/selenium/webdriver/remote/webdriver.py", line 413, in find_element_by_class_name
    return self.find_element(by=By.CLASS_NAME, value=name)
  File "/home/jaime/anaconda3/lib/python3.6/site-packages/selenium/webdriver/remote/webdriver.py", line 752, in find_element
    'value': value})['value']
  File "/home/jaime/anaconda3/lib/python3.6/site-packages/selenium/webdriver/remote/webdriver.py", line 236, in execute
    self.error_handler.check_response(response)
  File "/home/jaime/anaconda3/lib/python3.6/site-packages/selenium/webdriver/remote/errorhandler.py", line 192, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.NoSuchElementException: Message: no such element: Unable to locate element: {"method":"class name","se

No contract data per_contrata in http://www.gobiernotransparentechile.cl/directorio/entidad/1/1


could not get pagination from http://www.interior.gov.cl/transparenciahistorico/sag/dotacionplanta.html
could not get breadcrumbs from http://www.interior.gov.cl/transparenciahistorico/sag/dotacionplanta.html
could not get pagination from http://www.interior.gov.cl/transparenciahistorico/sag/dotacioncontrata.html
could not get breadcrumbs from http://www.interior.gov.cl/transparenciahistorico/sag/dotacioncontrata.html

could not get pagination from http://www.subdere.gov.cl/transparencia/dotacion_2008.html
could not get breadcrumbs from http://www.subdere.gov.cl/transparencia/dotacion_2008.html
could not get pagination from http://www.subdere.gov.cl/transparencia/dotacion_2007.html
could not get breadcrumbs from http://www.subdere.gov.cl/transparencia/dotacion_2007.html
could not get pagination from http://www.subdere.gov.cl/transparencia/dotacion_2006.html
could not get breadcrumbs from h

Traceback (most recent call last):
  File "./lib/functions.py", line 62, in getDepartmentData
    div_years = browser.find_element_by_class_name("linksIntermedios")
  File "/home/jaime/anaconda3/lib/python3.6/site-packages/selenium/webdriver/remote/webdriver.py", line 413, in find_element_by_class_name
    return self.find_element(by=By.CLASS_NAME, value=name)
  File "/home/jaime/anaconda3/lib/python3.6/site-packages/selenium/webdriver/remote/webdriver.py", line 752, in find_element
    'value': value})['value']
  File "/home/jaime/anaconda3/lib/python3.6/site-packages/selenium/webdriver/remote/webdriver.py", line 236, in execute
    self.error_handler.check_response(response)
  File "/home/jaime/anaconda3/lib/python3.6/site-packages/selenium/webdriver/remote/errorhandler.py", line 192, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.NoSuchElementException: Message: no such element: Unable to locate element: {"method":"class name","se

No contract data per_planta in http://www.gobiernotransparentechile.cl/directorio/entidad/2/4


...

No contract data per_contrata in http://www.gobiernotransparentechile.cl/directorio/entidad/19/269

could not get pagination from http://www.gobiernotransparentechile.cl/directorio/entidad/19/271/per_planta/A%C3%B1o%202017/A%C3%B1o%202017%20-%20Enero
could not get pagination from http://www.gobiernotransparentechile.cl/directorio/entidad/19/271/per_planta/A%C3%B1o%202017/A%C3%B1o%202017%20-%20Febrero
could not get pagination from http://www.gobiernotransparentechile.cl/directorio/entidad/19/271/per_planta/A%C3%B1o%202016/A%C3%B1o%202016%20-%20Enero
could not get pagination from http://www.gobiernotransparentechile.cl/directorio/entidad/19/271/per_planta/A%C3%B1o%202016/A%C3%B1o%202016%20-%20Febrero
could not get pagination from http://www.gobiernotransparentechile.cl/directorio/entidad/19/271/per_planta/A%C3%B1o%202016/A%C3%B1o%202016%20-%20Marzo
could not get pagination from http://www.gobiernotransparentechile.cl/directorio/entidad/19/271/per_planta/A%C3%B1o%202016/A%C3%B1o%202016%20

Traceback (most recent call last):
  File "./lib/functions.py", line 62, in getDepartmentData
    div_years = browser.find_element_by_class_name("linksIntermedios")
  File "/home/jaime/anaconda3/lib/python3.6/site-packages/selenium/webdriver/remote/webdriver.py", line 413, in find_element_by_class_name
    return self.find_element(by=By.CLASS_NAME, value=name)
  File "/home/jaime/anaconda3/lib/python3.6/site-packages/selenium/webdriver/remote/webdriver.py", line 752, in find_element
    'value': value})['value']
  File "/home/jaime/anaconda3/lib/python3.6/site-packages/selenium/webdriver/remote/webdriver.py", line 236, in execute
    self.error_handler.check_response(response)
  File "/home/jaime/anaconda3/lib/python3.6/site-packages/selenium/webdriver/remote/errorhandler.py", line 192, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.NoSuchElementException: Message: no such element: Unable to locate element: {"method":"class name","se

No contract data per_planta in http://www.gobiernotransparentechile.cl/directorio/entidad/21/270


Traceback (most recent call last):
  File "./lib/functions.py", line 62, in getDepartmentData
    div_years = browser.find_element_by_class_name("linksIntermedios")
  File "/home/jaime/anaconda3/lib/python3.6/site-packages/selenium/webdriver/remote/webdriver.py", line 413, in find_element_by_class_name
    return self.find_element(by=By.CLASS_NAME, value=name)
  File "/home/jaime/anaconda3/lib/python3.6/site-packages/selenium/webdriver/remote/webdriver.py", line 752, in find_element
    'value': value})['value']
  File "/home/jaime/anaconda3/lib/python3.6/site-packages/selenium/webdriver/remote/webdriver.py", line 236, in execute
    self.error_handler.check_response(response)
  File "/home/jaime/anaconda3/lib/python3.6/site-packages/selenium/webdriver/remote/errorhandler.py", line 192, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.NoSuchElementException: Message: no such element: Unable to locate element: {"method":"class name","se

No contract data per_contrata in http://www.gobiernotransparentechile.cl/directorio/entidad/21/270

could not get pagination from http://www.gobiernotransparentechile.cl/directorio/entidad/21/273/per_planta/Hist%C3%B3rico/A%C3%B1o%202006
could not get pagination from http://www.gobiernotransparentechile.cl/directorio/entidad/21/273/per_planta/Hist%C3%B3rico/A%C3%B1o%202007
could not get pagination from http://www.gobiernotransparentechile.cl/directorio/entidad/21/273/per_planta/Hist%C3%B3rico/A%C3%B1o%202008
could not get pagination from http://www.gobiernotransparentechile.cl/directorio/entidad/21/273/per_planta/Hist%C3%B3rico/A%C3%B1o%202009
could not get pagination from http://www.gobiernotransparentechile.cl/directorio/entidad/21/273/per_planta/Hist%C3%B3rico/A%C3%B1o%202010
could not get pagination from http://www.gobiernotransparentechile.cl/directorio/entidad/21/273/per_planta/Hist%C3%B3rico/A%C3%B1o%202011
could not get pagination from http://www.gobiernotransparentechile.cl/di

# Debugging


In [6]:
browser_debug = webdriver.Firefox()
getTableData2('./output/debugging.csv', 'http://www.gobiernotransparentechile.cl/directorio/entidad/12/167/per_planta/A%C3%B1o%202016/Enero', browser_debug)